# Basic use demo
This jupyter notebook shows the basic use of the internals.

#### Imports

In [1]:
import textwrap
import tempfile

from pathlib import Path

from file_handling.pipeline_input_handling.pipeline_input_file_manager import PipelineInputFileManager
from pipeline_entities.pipeline_configuration.dataclasses.pipeline_configuration_data import PipelineConfigurationData
from file_handling.pipeline_configuration_handling.pipeline_configuration_file_manager import \
    PipelineConfigurationFileManager
from pipeline_entities.pipeline_configuration.dataclasses.pipeline_configuration import PipelineConfiguration
from pipeline_entities.pipeline_input.pipeline_input import PipelineInput
from pipeline_entities.pipeline_input.pipeline_input_data import PipelineInputData
from pipeline_entities.pipeline.pipeline import Pipeline
from pipeline_entities.pipeline_builder.pipeline_builder import PipelineBuilder
from pipeline_entities.pipeline_manager.pipeline_manager import PipelineManager
from setup_manager.internal_logic_setup_manager import InternalLogicSetupManager

#### Setup

In [2]:
InternalLogicSetupManager.setup()

#### Pipeline configuration
A pipeline configuration defines the structure of a pipeline. Pipeline configurations are loaded from .ini files.
We are using a temporary file to make things easier for this demo.
The .ini content must contain the keys 'supported_program_version' with a value of the type 'packaging.Version' and 'components' with a value of the type 'Tree[str]'.
Values can be written over multiple lines by ending a line with the known '\\'.
We are using the following .ini content:

In [3]:
pipeline_configuration_file_content: bytes = textwrap.dedent("""\
    name="DemoPipeline"
    supported_program_version=Version(\"1.0.0\")
    components=Tree(\"\"\"\\
    BaseInput\\
         Equidistant Node Generator\\
           SingleFunctionExpressionInput\\
            Newton Interpolation\\
         Chebyshev1 Node Generator\\
         Chebyshev2 Node Generator\\
    \"\"\")
    extra_value=True
    """).encode("utf-8")

The pipeline configure has the name 'DemoPipeline', supports the program version 1.0.0 and has an additional entry with the key 'extra_value'.
It's most import entry are the components specified by the matching key named 'components'. Components are arranged in a tree.
The tree structure defines the order of component execution later on.
For easier use, the components are specified through their respective ids (e.g. the BaseInputPipelineComponent has the id 'BaseInput').


After defining the .ini file content, we write the content into a temporary file:

In [4]:
temp_dir = tempfile.TemporaryDirectory()
temp_pipeline_configuration_file = Path(temp_dir.name + "/pipeline_configuration.ini")

with open(temp_pipeline_configuration_file, "wb") as f:
    f.write(pipeline_configuration_file_content)

print(temp_dir.name)

C:\Users\49157\AppData\Local\Temp\tmplt346co9


With a .ini configuration file, we can start to read in the pipeline configuration into internal structures using the static class 'PipelineConfigurationManager'.
The operation 'load_from_file(path: Path)' reads in a .ini file and parses the input into an instance of the dataclass 'PipelineConfigurationData'.
Instances of this dataclass store the individual entries of a pipeline configuration .ini file as raw strings.
In particular, the values of entries are not evaluated (i.e. 'supported_program_version=Version("1.0.0")' will be stored the string "Version("1.0.0")" and not as an instance of the type 'packaging.Version').

In [5]:
pipeline_configuration_data: PipelineConfigurationData = PipelineConfigurationFileManager.load_from_file(temp_pipeline_configuration_file)

print(pipeline_configuration_data)

PipelineConfigurationData(name='"DemoPipeline"', supported_program_version='Version("1.0.0")', components='Tree("""\nBaseInput\n     Equidistant Node Generator\n       SingleFunctionExpressionInput\n        Newton Interpolation\n     Chebyshev1 Node Generator\n     Chebyshev2 Node Generator\n""")', additional_values={'extra_value': 'True'})


The last missing part is the parsing of the values. This can be conveniently done by simply instantiating an instance of the class 'PipelineConfiguration' using our pipeline_configuration_data.
Furthermore, the class 'PipelineConfiguration' will automatically check the types of all values and throw exceptions if given an entry with a key of wrong type (e.g. name is an int instead of a str).

In [6]:
pipeline_configuration: PipelineConfiguration = PipelineConfiguration(pipeline_configuration_data)

print(pipeline_configuration)

PipelineConfiguration(name='DemoPipeline', supported_program_version=<Version('1.0.0')>, components=Tree(root_node=TreeNode(value=PipelineComponentInfo(component_id='BaseInput', component_type=<class 'pipeline_entities.components.abstracts.input_pipeline_component.InputPipelineComponent'>, component_class=<class 'pipeline_entities.components.default_components.default_input_components.base_input_pipeline_component.BaseInputPipelineComponent'>, component_meta_info=ComponentMetaInfo(attributes_modifying={'interpolation_interval', 'node_count', 'data_type'}, dynamic_constraints=[], static_constraints=[], mixed_constraints=[])), child_nodes=[TreeNode(value=PipelineComponentInfo(component_id='equidistant node generator', component_type=<class 'pipeline_entities.components.abstracts.node_generator.NodeGenerator'>, component_class=<class 'pipeline_entities.components.default_components.default_node_generators.equidistant_node_generator.EquidistantNodeGenerator'>, component_meta_info=Component

#### PipelineInput
A pipeline input contains all the data used as input when executing the pipeline.
The pipeline input works very similarly to the pipeline configuration.
Required are the keys 'data_type' with a value of type 'type', 'node_count' with an int value and 'interpolation_interval' with a jax.numpy.ndarray of shape '(2,)' as the value.

In [7]:
pipeline_input_file_content: bytes = textwrap.dedent("""\
    name="TestPipeline"
    data_type=jax.numpy.float32
    node_count=5
    interpolation_interval=jax.numpy.array([-1, 1])
    single_function_expression="x**2 + 1"
    multiple_function_expressions=[((0,1), 'x'), ((1,2), 'x**2')]
    sympy_function_expression_simplification=True
    function_callable=lambda x: x**2 + 3
    function_values=jax.numpy.array([0.0, 1.0, 4.0, 9.0, 16.0])
    §secret_token="abc123"
    extra_value=[1, 2, 3]
    """).encode("utf-8")

We write the content into a temporary file:

In [8]:
temp_pipeline_input_file = Path(temp_dir.name + "/pipeline_input.ini")

with open(temp_pipeline_input_file, "wb") as f:
    f.write(pipeline_input_file_content)

Similiar to before, we can load the .ini file into an instance of the dataclass 'PipelineInputData' using the method 'load_from_file(path: Path)' from the static class 'PipelineInputFileManager'.

In [9]:
pipeline_input_data: PipelineInputData = PipelineInputFileManager.load_from_file(temp_pipeline_input_file)

print(pipeline_input_data)

PipelineInputData(name='"TestPipeline"', data_type='jax.numpy.float32', node_count='5', interpolation_interval='jax.numpy.array([-1, 1])', single_function_expression='"x**2 + 1"', multiple_function_expressions="[((0,1), 'x'), ((1,2), 'x**2')]", sympy_function_expression_simplification='True', function_callable='lambda x: x**2 + 3', function_values='jax.numpy.array([0.0, 1.0, 4.0, 9.0, 16.0])', additional_directly_injected_values={'secret_token': '"abc123"'}, additional_values={'extra_value': '[1, 2, 3]'})


We parse the 'pipeline_input_data' by initializing an instance of the class 'PipelineInput':

In [10]:
pipeline_input: PipelineInput = PipelineInput(pipeline_input_data)

print(pipeline_input)

PipelineInput(name='TestPipeline', data_type='<class 'jax.numpy.float32'>', node_count='5', interpolation_interval='[-1  1]', single_function_expression='x**2 + 1', multiple_function_expressions='[((0, 1), 'x'), ((1, 2), 'x**2')]', sympy_function_expression_simplification='True', function_callable='<function <lambda> at 0x0000017768D5CAE0>', function_values='[ 0.  1.  4.  9. 16.]', additional_directly_injected_values='{'secret_token': 'abc123'}', additional_values='{'extra_value': [1, 2, 3]}')


#### Pipeline building
Using the just created 'PipelineInput' and 'PipelineConfiguration' instances, we can assemble a pipeline:

In [11]:
pipeline: Pipeline = PipelineBuilder.build(pipeline_configuration, pipeline_input)

#### Pipeline execution
Instances of the class 'PipelineManager' are responsible for the execution of pipelines.
We start of by initializing a 'PipelineManager' with our pipeline.

In [12]:
pipeline_manager: PipelineManager = PipelineManager(pipeline)

After the initialization, we can execute the complete pipeline by calling 'execute_all()'.

In [13]:
pipeline_manager.execute_all()

# TODO: change
print(pipeline_manager._pipeline_data_dict_["/0/0/0/"])

PipelineData(data_type=<class 'jax.numpy.float32'>, node_count=5, interpolation_interval=Array([-1,  1], dtype=int32), function_values=Array([2.  , 1.25, 1.  , 1.25, 2.  ], dtype=float32), nodes=Array([-1. , -0.5,  0. ,  0.5,  1. ], dtype=float32), interpolant=Array([ 2. , -1.5,  1. ,  0. ,  0. ], dtype=float32), additional_values={'secret_token': 'abc123'})
